# Module 2: Dataframe Analysis

In [ ]:
import pandas as pd
import numpy as np
import time
import requests
from io import StringIO

### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

In [ ]:
url = "https://stockanalysis.com/ipos/withdrawn/"

headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
}

try:
  response = requests.get(url, headers=headers, timeout=10)
  response.raise_for_status()


  html_io = StringIO(response.text)
  withdrawn_IPO = pd.read_html(html_io)

  if not withdrawn_IPO:
    raise ValueError(f"No table found.")

except requests.exceptions.RequestException as e:
  print(f"Request failed: {e}")
except ValueError as ve:
  print(f"Data error: {ve}")
except Exception as ex:
  print(f"Unexpected error: {ex}")


withdrawn_IPO = withdrawn_IPO[0]

withdrawn_IPO.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [ ]:
withdrawn_IPO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [ ]:
# Regex-based classification function
import re

def classify_company(name):
    name = name.lower()
    if re.search(r'\bacquisition (corp|corporation)\b', name):
        return 'Acq.Corp'
    elif re.search(r'\binc\b|incorporated', name):
        return 'Inc'
    elif re.search(r'\bgroup\b', name):
        return 'Group'
    elif re.search(r'\bltd\b|limited', name):
        return 'Limited'
    elif re.search(r'\bholdings\b', name):
        return 'Holdings'
    else:
        return 'Other'

withdrawn_IPO['Company Class'] = withdrawn_IPO['Company Name'].apply(classify_company)

withdrawn_IPO.head(10)

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,Limited
6,CABR,"Caring Brands, Inc.",$4.00,750000,Inc
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000,Inc
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000,Limited
9,KMCM,Key Mining Corp.,$2.25,4444444,Other


In [ ]:
# Parse and calculate average price
def parse_price_range(price):
    if price == '-' or pd.isna(price):
        return None
    matches = re.findall(r'\$?(\d+(?:\.\d+)?)', price)
    if not matches:
        return None
    values = [float(p) for p in matches]
    return sum(values) / len(values)

withdrawn_IPO['Avg. price'] = withdrawn_IPO['Price Range'].apply(parse_price_range)

withdrawn_IPO.head(10)

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.50
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000,Limited,5.00
6,CABR,"Caring Brands, Inc.",$4.00,750000,Inc,4.00
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000,Inc,9.00
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000,Limited,5.00
9,KMCM,Key Mining Corp.,$2.25,4444444,Other,2.25


In [ ]:
# Clean 'Shares Offered' column

withdrawn_IPO['Shares Offered'] = withdrawn_IPO['Shares Offered'].astype(str).str.replace('[^0-9.-]', '', regex=True)
withdrawn_IPO['Shares Offered'] = pd.to_numeric(withdrawn_IPO['Shares Offered'], errors='coerce')

In [ ]:
withdrawn_IPO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          100 non-null    object 
 1   Company Name    100 non-null    object 
 2   Price Range     100 non-null    object 
 3   Shares Offered  72 non-null     float64
 4   Company Class   100 non-null    object 
 5   Avg. price      73 non-null     float64
dtypes: float64(2), object(4)
memory usage: 4.8+ KB


In [ ]:
# New Withdrawn Value column
withdrawn_IPO['Withdrawn Value'] = withdrawn_IPO['Shares Offered'] * withdrawn_IPO['Avg. price']

In [ ]:
# Group by Company Class and calculate total withdrawn value

ipo_value_by_class = withdrawn_IPO.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False)

# Find the company class with the highest total withdrawal value
highest_value_class = ipo_value_by_class.index[0]
highest_total_value = ipo_value_by_class.iloc[0] / 1_000_000

print(f"\nCompany class with the highest total withdrawal value: {highest_value_class}")
print(f"Total withdrawn value for {highest_value_class}: ${highest_total_value:.2f} million")


Company class with the highest total withdrawal value: Acq.Corp
Total withdrawn value for Acq.Corp: $4021.00 million


In [ ]:
ipo_value_by_class.head()

,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Inc,2.257164e+09
Other,7.679200e+08
Limited,5.497346e+08
Holdings,7.500000e+07


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.

In [ ]:
url = "https://stockanalysis.com/ipos/2024/"

headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
}

try:
  response = requests.get(url, headers=headers, timeout=10)
  response.raise_for_status()


  html_io = StringIO(response.text)
  IPO_2024 = pd.read_html(html_io)

  if not IPO_2024:
    raise ValueError(f"No table found.")

except requests.exceptions.RequestException as e:
  print(f"Request failed: {e}")
except ValueError as ve:
  print(f"Data error: {ve}")
except Exception as ex:
  print(f"Unexpected error: {ex}")


IPO_2024 = IPO_2024[0]

IPO_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$5.00,25.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$26.57,564.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.62,-84.40%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.28,2.80%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.21,2.10%


In [ ]:
IPO_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [ ]:
# Filter IPO_2024 to before 1 June 2024 (first 5 months)

IPO_2024['IPO Date'] = pd.to_datetime(IPO_2024['IPO Date'])
first_IPO_2024 = IPO_2024[IPO_2024['IPO Date'] < '2024-06-01'].copy()
first_IPO_2024.reset_index(drop=True, inplace=True)
first_IPO_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.22,-
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.49,114.65%
2,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$18.70,-4.40%
3,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.60,6.00%
4,2024-05-15,JDZG,JIADE Limited,$4.00,$0.26,-93.40%


In [ ]:
first_IPO_2024.replace('-', pd.NA, inplace=True)

In [ ]:
first_IPO_2024['IPO Price'] = first_IPO_2024['IPO Price'].astype(str).str.replace('$', '', regex=False)
first_IPO_2024['IPO Price'] = pd.to_numeric(first_IPO_2024['IPO Price'], errors='coerce')

first_IPO_2024['Current'] = first_IPO_2024['Current'].astype(str).str.replace('$', '', regex=False)
first_IPO_2024['Current'] = pd.to_numeric(first_IPO_2024['Current'], errors='coerce')

first_IPO_2024['Return'] = first_IPO_2024['Return'].astype(str).str.replace('%', '', regex=False)
first_IPO_2024['Return'] = pd.to_numeric(first_IPO_2024['Return'], errors='coerce')

In [ ]:
first_IPO_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      77 non-null     datetime64[ns]
 1   Symbol        77 non-null     object        
 2   Company Name  77 non-null     object        
 3   IPO Price     75 non-null     float64       
 4   Current       77 non-null     float64       
 5   Return        75 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 3.7+ KB


In [ ]:
#first_IPO_2024.dropna(inplace=True)
#first_IPO_2024.info()

In [ ]:
import yfinance as yf

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(first_IPO_2024.Symbol):

  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90, 252, 365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  # historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  # historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  # historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  # historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  # historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

In [ ]:
stocks_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,growth_future_30d,volatility
0,4.00,4.20,2.80,3.02,440600,0.0,0.0,NAKA,2024-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.784768,NaN
1,2.99,3.11,2.35,2.66,147300,0.0,0.0,NAKA,2024-06-03,0.880795,NaN,NaN,NaN,NaN,NaN,NaN,0.883459,NaN
2,2.53,3.11,2.41,2.92,73800,0.0,0.0,NAKA,2024-06-04,1.097744,NaN,NaN,NaN,NaN,NaN,NaN,0.955479,NaN
3,2.91,3.09,2.60,2.73,51100,0.0,0.0,NAKA,2024-06-05,0.934931,0.903974,NaN,NaN,NaN,NaN,NaN,1.051282,NaN
4,2.94,2.94,2.41,2.69,56500,0.0,0.0,NAKA,2024-06-06,0.985348,1.011278,NaN,NaN,NaN,NaN,NaN,1.111524,NaN


In [ ]:
# Calculate the Sharpe ratio assuming a risk-free rate of 4.5%
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [ ]:
# Filter the DataFrame to keep data only for the trading day: 2025-06-06

stocks_df_day = stocks_df[stocks_df['Date'] == pd.to_datetime('2025-06-06').date()].copy()
stocks_df_day.reset_index(drop=True, inplace=True)
stocks_df_day.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,growth_future_30d,volatility,Sharpe
0,16.260000,17.200001,15.550000,15.880000,431200,0.0,0.0,NAKA,2025-06-06,0.991261,0.864924,0.655386,9.286550,11.762963,5.438356,NaN,NaN,136.860031,0.039408
1,36.439999,36.540001,35.930000,36.389999,137900,0.0,0.0,BOW,2025-06-06,1.010272,0.971696,0.996986,0.879623,1.105407,1.442331,NaN,NaN,24.508713,0.057014
2,20.430000,20.440001,19.969999,20.410000,1300,0.0,0.0,HDL,2025-06-06,0.998532,1.023160,1.039206,0.969366,0.869992,1.007155,NaN,NaN,18.816306,0.051134
3,10.510000,10.510000,10.510000,10.510000,0,0.0,0.0,RFAI,2025-06-06,1.000000,1.000952,1.002863,1.007960,1.024366,NaN,NaN,NaN,0.356043,NaN
4,0.380000,0.386000,0.201000,0.295000,3428900,0.0,0.0,JDZG,2025-06-06,0.862573,0.565134,0.556604,0.564054,0.393858,0.168571,NaN,NaN,1.475159,0.083768


In [ ]:
stocks_df_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Open               77 non-null     float64
 1   High               77 non-null     float64
 2   Low                77 non-null     float64
 3   Close              77 non-null     float64
 4   Volume             77 non-null     int64  
 5   Dividends          77 non-null     float64
 6   Stock Splits       77 non-null     float64
 7   Ticker             77 non-null     object 
 8   Date               77 non-null     object 
 9   growth_1d          77 non-null     float64
 10  growth_3d          77 non-null     float64
 11  growth_7d          77 non-null     float64
 12  growth_30d         77 non-null     float64
 13  growth_90d         77 non-null     float64
 14  growth_252d        73 non-null     float64
 15  growth_365d        0 non-null      float64
 16  growth_future_30d  0 non-nul

In [ ]:
print('growth_252d:')
print(stocks_df_day['growth_252d'].describe())

print('\nSharpe:')
print(stocks_df_day['Sharpe'].describe())

growth_252d:
count    73.000000
mean      1.227946
std       1.480238
min       0.024970
25%       0.293510
50%       0.763188
75%       1.446667
max       8.097413
Name: growth_252d, dtype: float64

Sharpe:
count    73.000000
mean      0.284576
std       0.512601
min      -0.079677
25%       0.040265
50%       0.083768
75%       0.291048
max       2.835668
Name: Sharpe, dtype: float64


In [ ]:
print('Median Sharpe Ratio:', stocks_df_day['Sharpe'].median())

Median Sharpe Ratio: 0.08376823003294499


In [ ]:
# Top 10 by growth_252d
top_10_growth = stocks_df_day.nlargest(10, 'growth_252d')[['Ticker', 'growth_252d']]
print("\nTop 10 tickers by growth_252d:")
print(top_10_growth)

# Top 10 by Sharpe
top_10_sharpe = stocks_df_day.nlargest(10, 'Sharpe')[['Ticker', 'Sharpe']]
print("\nTop 10 tickers by Sharpe:")
print(top_10_sharpe)


Top 10 tickers by growth_252d:
   Ticker  growth_252d
68     JL     8.097413
76   ROMA     6.156406
0    NAKA     5.438356
50   UMAC     4.966533
11    NNE     4.655224
17   RBRK     3.184065
57    AHR     2.483097
61     AS     2.478203
26   SUPX     2.346065
16    MRX     2.300384

Top 10 tickers by Sharpe:
   Ticker    Sharpe
39   BKHA  2.835668
70   JVSA  2.041531
58   LEGT  1.940267
37   IBAC  1.637119
51   HLXB  1.123493
30   MNDR  0.974234
48   DYCQ  0.969321
43   INTJ  0.744512
68     JL  0.566222
21   TRSG  0.518080


### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

In [ ]:
new_stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(first_IPO_2024.Symbol):

  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in range(1,13):
    historyPrices['future_growth_'+str(i)+'m'] = historyPrices['Close'].shift(-(i*21)) / historyPrices['Close']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if new_stocks_df.empty:
    new_stocks_df = historyPrices
  else:
    new_stocks_df = pd.concat([new_stocks_df, historyPrices], ignore_index=True)

In [ ]:
new_stocks_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,future_growth_1m,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,4.00,4.20,2.80,3.02,440600,0.0,0.0,NAKA,2024-05-31,0.728477,...,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,2.99,3.11,2.35,2.66,147300,0.0,0.0,NAKA,2024-06-03,0.924812,...,0.439850,0.390226,0.379699,0.428571,0.529323,0.454887,0.642857,0.556391,1.150376,6.022556
2,2.53,3.11,2.41,2.92,73800,0.0,0.0,NAKA,2024-06-04,0.770548,...,0.434931,0.349315,0.332192,0.541096,0.523973,0.414384,0.565068,0.489726,1.058219,5.438356
3,2.91,3.09,2.60,2.73,51100,0.0,0.0,NAKA,2024-06-05,0.824176,...,0.487179,0.378388,0.340659,0.648352,0.511722,0.449084,0.564103,0.597070,1.384615,4.890110
4,2.94,2.94,2.41,2.69,56500,0.0,0.0,NAKA,2024-06-06,0.910781,...,0.496283,0.405204,0.334572,0.620818,0.609665,0.464684,0.527881,0.650558,1.449814,4.312268


In [ ]:
new_stocks_df['Date'] = pd.to_datetime(new_stocks_df['Date'])

# Group by Ticker and get the earliest date for each
min_dates = new_stocks_df.groupby('Ticker')['Date'].min().reset_index()

min_dates.rename(columns={'Date': 'min_date'}, inplace=True)

In [ ]:
# Inner join on both Ticker and Date = min_date
min_date_stocks = pd.merge(
    new_stocks_df,
    min_dates,
    left_on=['Ticker', 'Date'],
    right_on=['Ticker', 'min_date'],
    how='inner'
)

min_date_stocks.drop(columns='min_date', inplace=True)
min_date_stocks.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Date,future_growth_1m,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,4.0,4.20,2.800000,3.020000,440600,0.0,0.0,NAKA,2024-05-31,0.728477,...,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,23.0,24.27,22.139999,23.799999,3335800,0.0,0.0,BOW,2024-05-23,1.024790,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
2,27.0,30.00,19.799999,22.290001,670000,0.0,0.0,HDL,2024-05-17,0.775236,...,0.673845,0.643786,0.720502,0.719605,1.255271,1.106326,1.030507,1.192014,0.907133,1.002602
3,10.0,10.08,10.000000,10.040000,1175800,0.0,0.0,RFAI,2024-07-05,1.002191,...,1.007968,1.014940,1.014940,1.017928,1.022908,1.030877,1.035857,1.040837,1.046813,NaN
4,4.8,5.40,3.595000,4.070000,2366200,0.0,0.0,JDZG,2024-05-15,0.242998,...,0.122850,0.160442,0.272727,0.199017,0.139066,0.182555,0.170025,0.146437,0.121867,0.142506


In [ ]:
min_date_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Open               77 non-null     float64       
 1   High               77 non-null     float64       
 2   Low                77 non-null     float64       
 3   Close              77 non-null     float64       
 4   Volume             77 non-null     int64         
 5   Dividends          77 non-null     float64       
 6   Stock Splits       77 non-null     float64       
 7   Ticker             77 non-null     object        
 8   Date               77 non-null     datetime64[ns]
 9   future_growth_1m   77 non-null     float64       
 10  future_growth_2m   77 non-null     float64       
 11  future_growth_3m   77 non-null     float64       
 12  future_growth_4m   77 non-null     float64       
 13  future_growth_5m   77 non-null     float64       
 14  future_growt

In [ ]:
selected_columns = [
    'future_growth_1m', 'future_growth_2m', 'future_growth_3m', 'future_growth_4m',
    'future_growth_5m', 'future_growth_6m', 'future_growth_7m', 'future_growth_8m',
    'future_growth_9m', 'future_growth_10m', 'future_growth_11m', 'future_growth_12m'
]

summary_stats = new_stocks_df[selected_columns].describe()

summary_stats

,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,22263.000000,20646.000000,19029.000000,17412.000000,15795.000000,14178.000000,12561.000000,10944.000000,9327.000000,7726.000000,6130.000000,4556.000000
mean,1.042045,1.073601,1.087384,1.111854,1.131749,1.150257,1.148120,1.138648,1.119703,1.088825,1.038416,0.989229
std,0.822013,1.100744,1.201256,1.340014,1.449729,1.586753,1.551796,1.525979,1.475659,1.385588,1.232480,1.152972
min,0.020233,0.011224,0.008863,0.013852,0.014815,0.010148,0.009901,0.010849,0.009314,0.012727,0.012235,0.019012
25%,0.853964,0.763533,0.688067,0.622665,0.575005,0.515639,0.467280,0.413586,0.370492,0.345265,0.307368,0.266238
50%,0.994236,0.972417,0.949547,0.931231,0.915479,0.908665,0.908683,0.861393,0.814636,0.762745,0.709579,0.656379
75%,1.081512,1.100000,1.116655,1.128534,1.165861,1.193031,1.234921,1.266541,1.261539,1.220030,1.159932,1.117973
max,33.944955,23.888887,21.764706,19.230159,22.348215,28.022473,23.975226,20.349594,16.395665,17.857143,12.833676,10.945710


In [ ]:
mean_growth = new_stocks_df[selected_columns].mean()
max_value = mean_growth.max()
max_value

1.1502566409551316

In [ ]:
# Months where the mean is equal to the max value
max_months = mean_growth[round(mean_growth,2) >= 1.15].index

# Mean of all other months (excluding max)
other_months = mean_growth[~mean_growth.index.isin(max_months)]
mean_of_others = other_months.mean()

uplift_percent = ((max_value - mean_of_others) / mean_of_others) * 100

print(f"Max mean future growth: {max_value:.2f}")
print(f"Mean of other months: {mean_of_others:.2f}")
print(f"Uplift: {uplift_percent:.2f}%")

Max mean future growth: 1.15
Mean of other months: 1.08
Uplift: 6.29%


In [ ]:
len(max_months)

2

### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in \$ thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---

In [156]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=c34de267-8653-4d1e-97d8-eba229273159
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 96.3MB/s]


In [157]:
df.head()

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
df.rsi

,rsi
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5686,68.312972
5687,67.123448
5688,67.563912
5689,68.218963


**Calculate Net Profit Over 25 Years**

In [163]:
rsi_threshold = 25 # for oversold
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [164]:
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1568 entries, 3668 to 4409
Columns: 203 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 2.1+ MB


In [165]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [166]:
net_income_k = net_income / 1000  # convert to $K
print(f"Net income from RSI-based strategy (2000–2025): ${net_income_k:.2f}K")

Net income from RSI-based strategy (2000–2025): $24.30K
